In [36]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from soilgasflux_fcs import multiprocess_raw_data, json_reader, fcs
from soilgasflux_fcs import synthetic_postProcessing
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
# /Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-24
data_folder = pathlib.Path(r'/Users/alexnaokiasatokobayashi/Documents/data/Gals/gals_20250829/sgf_07_pt02/data/2025-08-23')
# data_folder = pathlib.Path(r'/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-23')
a = json_reader.Initializer(folderPath=data_folder)
df = a.prepare_rawdata()

In [99]:
df.dtypes

datetime_utc               object
si_temperature            float64
bmp_temperature           float64
si_humidity               float64
datetime           datetime64[ns]
k30_co2                     int64
bmp_pressure              float64
id                         object
timedelta                   int64
dtype: object

In [100]:
# fig, ax = plt.subplots()
dfs = []
for i in df['id'].unique():
    df_id = df.loc[df['id']==i]
    df_id = df_id.sort_values(by='timedelta')

    df_id.set_index('datetime', inplace=True)
    df_id = df_id.asfreq('S')

    df_id_interpolated = df_id.interpolate(method='linear')
    df_id_interpolated['id'] = df_id_interpolated['id'].fillna(method='ffill')
    df_id_interpolated.reset_index(inplace=True)
    dfs.append(df_id_interpolated)

    # print(df_id_interpolated)
    # break
df_interpolated = pd.concat(dfs)

In [101]:
for i in df_interpolated['id'].unique():
    df_id = df_interpolated.loc[df_interpolated['id']==i]
    

In [102]:
df_interpolated

,datetime,datetime_utc,si_temperature,bmp_temperature,si_humidity,k30_co2,bmp_pressure,id,timedelta
0,2025-08-23 00:00:32,2025-08-23 00:00:32,18.86268,21.180,75.05469,517.0,96969.27,2025-08-23_00-00-32,0.0
1,2025-08-23 00:00:33,NaN,18.86268,21.195,75.14624,517.5,96968.97,2025-08-23_00-00-32,1.0
2,2025-08-23 00:00:34,2025-08-23 00:00:34,18.86268,21.210,75.23779,518.0,96968.67,2025-08-23_00-00-32,2.0
3,2025-08-23 00:00:35,2025-08-23 00:00:35,18.86268,21.250,75.48193,519.0,96968.59,2025-08-23_00-00-32,3.0
4,2025-08-23 00:00:36,2025-08-23 00:00:36,18.85195,21.280,75.69556,519.0,96968.75,2025-08-23_00-00-32,4.0
...,...,...,...,...,...,...,...,...,...
135,2025-08-23 23:36:14,2025-08-23 23:36:14,15.91327,19.940,87.18542,601.0,97122.00,2025-08-23_23-33-59,135.0
136,2025-08-23 23:36:15,2025-08-23 23:36:15,15.91327,19.950,87.20068,602.0,97120.94,2025-08-23_23-33-59,136.0
137,2025-08-23 23:36:16,2025-08-23 23:36:16,15.90255,19.960,87.17017,603.0,97119.08,2025-08-23_23-33-59,137.0
138,2025-08-23 23:36:17,2025-08-23 23:36:17,15.92400,19.960,87.20068,603.0,97119.97,2025-08-23_23-33-59,138.0


In [103]:
df_interpolated.groupby(by='id')['timedelta'].max().min(),df_interpolated.groupby(by='id')['timedelta'].max().max()


(139.0, 141.0)

In [104]:
df_interpolated.loc[df_interpolated['timedelta']>df_interpolated.groupby(by='id')['timedelta'].max().min()] = np.nan
# df_interpolated.loc[df_interpolated['timedelta']>131] = np.nan

In [105]:
df_interpolated.dropna(subset='timedelta', inplace=True)

In [106]:
# df_interpolated.dropna(inplace=True)

In [107]:
df_interpolated['timedelta'].unique()

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139.])

In [108]:
df_interpolated['timedelta'] = df_interpolated['timedelta'].astype(int)

In [ ]:
a = multiprocess_raw_data.Multiprocessor()

b = a.run(df=df_interpolated, chamber_id='test01_20250823', output_folder=r'/Users/alexnaokiasatokobayashi/Documents/data/Gals/results')
# b = a.run(df=df_interpolated, chamber_id='test01_20250823', output_folder=r'/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/results')

Multiprocessing started
CPU core count: 14


Date: 2025-08-23
Processing ID: 2025-08-23_02-25-28
Processing ID: 2025-08-23_05-26-30
Processing ID: 2025-08-23_03-01-40
Processing ID: 2025-08-23_07-15-07
Processing ID: 2025-08-23_00-36-47
Processing ID: 2025-08-23_01-49-15
Processing ID: 2025-08-23_04-14-05
Processing ID: 2025-08-23_06-02-42
Processing ID: 2025-08-23_03-37-53
Processing ID: 2025-08-23_07-51-20
Processing ID: 2025-08-23_00-00-32
Processing ID: 2025-08-23_06-38-55
Processing ID: 2025-08-23_04-50-17
Processing ID: 2025-08-23_01-13-01
Results: dict_keys(['2025-08-23_04-50-17'])
Processing ID: 2025-08-23_05-16-01
Results: dict_keys(['2025-08-23_01-49-15'])
Processing ID: 2025-08-23_02-14-59
Results: dict_keys(['2025-08-23_02-14-59'])
Processing ID: 2025-08-23_02-20-13
Results: dict_keys(['2025-08-23_00-36-47'])
Processing ID: 2025-08-23_01-02-32
Results: dict_keys(['2025-08-23_06-02-42'])
Processing ID: 2025-08-23_06-28-26
Results: dict_keys(['2025-08-23_01-13-01'])
Processing ID: 2025-08-23_01-38-45
Results: dict_keys(

In [114]:
b = a.run_MC(df=df_interpolated, chamber_id='test8kMC_20250823', output_folder=r'/Users/alexnaokiasatokobayashi/Documents/data/Gals/results')

Multiprocessing started
CPU core count: 14
Date: 2025-08-23
Processing ID: 2025-08-23_01-13-01
Processing ID: 2025-08-23_02-25-28
Processing ID: 2025-08-23_03-01-40
Processing ID: 2025-08-23_00-36-47
Processing ID: 2025-08-23_07-15-07
Processing ID: 2025-08-23_01-49-15
Processing ID: 2025-08-23_04-14-05
Processing ID: 2025-08-23_05-26-30
Processing ID: 2025-08-23_04-50-17
Processing ID: 2025-08-23_03-37-53
Processing ID: 2025-08-23_06-02-42
Processing ID: 2025-08-23_00-00-32
Processing ID: 2025-08-23_06-38-55
Processing ID: 2025-08-23_07-51-20
Results: dict_keys(['2025-08-23_07-51-20'])
Processing ID: 2025-08-23_08-17-03
Results: dict_keys(['2025-08-23_03-01-40'])
Processing ID: 2025-08-23_03-27-24
Results: dict_keys(['2025-08-23_07-15-07'])
Processing ID: 2025-08-23_07-40-51
Results: dict_keys(['2025-08-23_06-02-42'])
Processing ID: 2025-08-23_06-28-26
Results: dict_keys(['2025-08-23_04-14-05'])
Processing ID: 2025-08-23_04-39-48
Results: dict_keys(['2025-08-23_00-36-47'])
Processing 

In [126]:
ds_test = xr.open_dataset(r'/Users/alexnaokiasatokobayashi/Documents/data/Gals/results/test8kMC_20250823_2025-08-23.nc')
test = a.select_bestPareto(ds=ds_test)

Selecting best pareto front from dataset
Best dcdt shape: (118, 8000)
